## Using GitLoader (Langchain) to load git repos.

In [17]:
from langchain.document_loaders import GitLoader
from langchain.schema import Document
import json
from typing import Iterable

In [18]:
# The original approach loads too many files. 
# Instead, focus on Tcl, Cpp files

cpp_exts = ['.cpp', 'cc', 'c++',
            'hpp', 'hh', 'h++', 'h']
other_exts = ['.tcl', '.i', '.py', '.md']
exts = cpp_exts + other_exts

loader = GitLoader(
    clone_url="https://github.com/The-OpenROAD-Project/OpenROAD",
    repo_path="./data",
    branch="master",
    file_filter = lambda file_path: any(file_path.endswith(ext) for ext in exts)
)

data = loader.load_and_split()
data

[Document(page_content="# Contributor Covenant Code of Conduct\n\n## Our Pledge\n\nWe as members, contributors, and leaders pledge to make participation in our\ncommunity a harassment-free experience for everyone, regardless of age, body\nsize, visible or invisible disability, ethnicity, sex characteristics, gender\nidentity and expression, level of experience, education, socio-economic status,\nnationality, personal appearance, race, religion, or sexual identity\nand orientation.\n\nWe pledge to act and interact in ways that contribute to an open, welcoming,\ndiverse, inclusive, and healthy community.\n\n## Our Standards\n\nExamples of behavior that contributes to a positive environment for our\ncommunity include:\n\n* Demonstrating empathy and kindness toward other people\n* Being respectful of differing opinions, viewpoints, and experiences\n* Giving and gracefully accepting constructive feedback\n* Accepting responsibility and apologizing to those affected by our mistakes,\n  and l

In [19]:
len(data)

7037

In [20]:
# Code is adapted from https://github.com/langchain-ai/langchain/issues/3016
def save_docs_to_jsonl(array:Iterable[Document], file_path:str)->None:
    with open(file_path, 'w') as jsonl_file:
        for doc in array:
            jsonl_file.write(doc.json() + '\n')

def load_docs_from_jsonl(file_path)->Iterable[Document]:
    array = []
    with open(file_path, 'r') as jsonl_file:
        for line in jsonl_file:
            data = json.loads(line)
            obj = Document(**data)
            array.append(obj)
    return array
    
save_docs_to_jsonl(data,'tempdata/data.jsonl')
data2=load_docs_from_jsonl('tempdata/data.jsonl')
assert len(data) == len(data2)
